In [1]:
import pandas as pd
from functools import reduce

In [2]:
annot_dir = '/stor/work/Marcotte/project/rmcox/leca/ppi_ml/annotations'
# complex annotations
cmplx_file = f'{annot_dir}/cmplx/all_cmplx_annots.csv'
# clade annotations
clade_file = f'{annot_dir}/leca_nogs_cov_fraction.csv'
# uniprot annotations
up_file = f'{annot_dir}/leca_eunog_annots.030721.csv'
# walktrap result file
old_res_file = '../ppi_ml/results/archive/ppi_clustering/final_cmplx_annots_011823.csv'
clst_file = '../ppi_ml/results/test_new_walktrap/walktrap_test_5steps.csv'

In [3]:
# load all annotations
cmplx = pd.read_csv(cmplx_file)
clade = pd.read_csv(clade_file)
uniprot = pd.read_csv(up_file)

In [4]:
clade = clade.rename(columns={'Amorphea':'amor_11', 'Archaeplastida':'viri_13', 'Excavate':'exca_2', 'TSAR':'tsar_5'})
clade = clade[['ID', 'amor_11', 'exca_2', 'tsar_5', 'viri_13']]

In [5]:
# include old annotations
old_res = pd.read_csv(old_res_file)
old_res_cmplx = old_res[['ID', 'characterization_status', 'granulated_cmplx_name']]
old_res_cmplx = old_res_cmplx.rename(columns={'characterization_status':'old_status', 'granulated_cmplx_name': 'old_cmplx_assignment'})
old_res_cmplx

,ID,old_status,old_cmplx_assignment
0,ENOG502S254,Uncharacterized,Uncharacterized UBL complex
1,KOG4146,Uncharacterized,Uncharacterized UBL complex
2,KOG1701,Uncharacterized,Uncharacterized endosomal trafficking complex
3,KOG2221,Uncharacterized,Uncharacterized endosomal trafficking complex
4,KOG2422,Known,Ribosome quality control complex 2
...,...,...,...
2773,KOG1207,Uncharacterized,Uncharacterized complex 20
2774,ENOG502QQEE,Uncharacterized,Uncharacterized complex 21
2775,KOG2925,Uncharacterized,Uncharacterized complex 21
2776,KOG0702,Known,Nuclear pore complex 4


In [6]:
# combine all into a single annotation file
annots = pd.DataFrame()
annots = pd.merge(clade, old_res_cmplx, how='left', on=['ID'])
annots = pd.merge(annots, cmplx, how='left', on=['ID'])
annots = pd.merge(annots, uniprot, how='left', on=['ID'])
annots.to_csv('../ppi_ml/annotations/leca_eunog_annots_complete.030721.csv', index=False)
    
# annot_df = reduce(lambda x, y: x.join(y, how='outer', on='ID'), annot_list)
# annot_df

In [7]:
# load data to annotate
data = pd.read_csv(clst_file)
data.columns

Index(['ID', 'cut_194', 'cut_388', 'cut_582', 'cut_776', 'cut_970', 'cut_1164',
       'cut_1358', 'cut_1552', 'cut_1746', 'cut_1940'],
      dtype='object')

In [8]:
df = pd.merge(data, clade, how='left', on=['ID'])
df = pd.merge(df, old_res_cmplx, how='left', on=['ID'])
df = pd.merge(df, cmplx, how='left', on=['ID'])
df = pd.merge(df, uniprot, how='left', on=['ID'])

In [9]:
basename = clst_file.split('.csv', 1)[0]
outname = basename+'_cmplx_annots'
df.to_csv(outname+'.csv', index=False)
df.to_excel(outname+'.xlsx', index=False)